<a href="https://colab.research.google.com/github/IcaroTheCreator-ufam/Sports-Injury-Predictions/blob/main/PCA/P%C3%B3s-PCA/P%C3%93S_PCA_REGRESS%C3%83O_LOG%C3%8DSTICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NFL Injury Analysis



In [ ]:
#Importando as bibliotecas necessárias para processamento de dados, plotagem gráfica e treinamento dos modelos

import numpy as np
import pandas as pd #processamento de dados, para abrir o dataset (arquivo .csv)
import itertools
import seaborn as sns
from sklearn.model_selection import train_test_split
!pip install mglearn

from google.colab import drive #para conectar o google drive ao notebook e trazer os datasets para cá
drive.mount('/content/drive') #diretório onde o drive vai ser instaurado

'''#from google.colab import files''' #função import para justamente fazer o upload dos arquivos do drive
'''uploaded = files.upload()''' #para caso de upload

# para caso de usar imagens
import PIL
from PIL import Image

# para plotagem de gráficos
import matplotlib.pyplot as plt #biblioteca mais comum para plotar as imagens
import seaborn as snsC
import scipy.stats as ss

# bibliotecas e modelos para o treinamento dos modelos
from sklearn.model_selection import StratifiedKFold #função da biblioteca sklearn para dividir os sets de maneira estratificada
from sklearn.model_selection import train_test_split #função da biblioteca sklearn para dividir os sets de maneira padrão
from imblearn.over_sampling import RandomOverSampler, SMOTE #funções para over sampling (aumentar quantidade de ex da classe minoritária (lesão))
import xgboost as xgb #um dos modelos de ML
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score #funções que irão avaliar os modelos por meio de scores
from sklearn.metrics import (ConfusionMatrixDisplay, f1_score, classification_report) #mais funções para avaliação do modelo
from sklearn.tree import export_graphviz #função para imprimir a árvore de decisão
import graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 5.2 MB/s eta 0:00:00
Mounted at /content/drive


# Load Data

In [ ]:
play_df = pd.read_csv('/content/drive/My Drive/Datasets/PlayList.csv')
#player_df = pd.read_csv('/content/drive/My Drive/Datasets/PlayerTrackData.csv')
#este último dataset foi descartado pois ele pesa 3.7 GB e estoura a memória RAM gratuita do Colab e também seus dados não afetam o treinamento de nossos modelos
injury_df = pd.read_csv('/content/drive/My Drive/Datasets/InjuryRecord.csv')

In [ ]:
#Exploração de dados
#Primeiramente gostaria de fazer a limpeza inicial e fazer visualizações para me familiarizar com os dados e gerar ideias para análises posteriores.
#Explorar conjunto de dados

unique_players = play_df.PlayerKey.nunique()
unique_games = play_df.GameID.nunique()
unique_plays = play_df.PlayKey.nunique()

print('There are {} players in the dataset.'.format(unique_players))
print('There are {} games in the dataset.'.format(unique_games))
print('There are {} plays in the dataset.'.format(unique_plays))

There are 250 players in the dataset.
There are 5712 games in the dataset.
There are 267005 plays in the dataset.


In [ ]:
#Game exploration:
# create a dataframe with game-level information
game_df = play_df[['GameID', 'StadiumType', 'FieldType', 'Weather', 'Temperature']].drop_duplicates().reset_index().drop(columns=['index'])

In [ ]:
def clean_weather(row):
    cloudy = ['Cloudy 50% change of rain', 'Hazy', 'Cloudy.', 'Overcast', 'Mostly Cloudy',
          'Cloudy, fog started developing in 2nd quarter', 'Partly Cloudy',
          'Mostly cloudy', 'Rain Chance 40%',' Partly cloudy', 'Party Cloudy',
          'Rain likely, temps in low 40s', 'Partly Clouidy', 'Cloudy, 50% change of rain','Mostly Coudy', '10% Chance of Rain',
          'Cloudy, chance of rain', '30% Chance of Rain', 'Cloudy, light snow accumulating 1-3"',
          'cloudy', 'Coudy', 'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
         'Cloudy fog started developing in 2nd quarter', 'Cloudy light snow accumulating 1-3"',
         'Cloudywith periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
         'Cloudy 50% change of rain', 'Cloudy and cold',
       'Cloudy and Cool', 'Partly cloudy']

    clear = ['Clear, Windy',' Clear to Cloudy', 'Clear, highs to upper 80s',
             'Clear and clear','Partly sunny',
             'Clear, Windy', 'Clear skies', 'Sunny', 'Partly Sunny', 'Mostly Sunny', 'Clear Skies',
             'Sunny Skies', 'Partly clear', 'Fair', 'Sunny, highs to upper 80s', 'Sun & clouds', 'Mostly sunny','Sunny, Windy',
             'Mostly Sunny Skies', 'Clear and Sunny', 'Clear and sunny','Clear to Partly Cloudy', 'Clear Skies',
            'Clear and cold', 'Clear and warm', 'Clear and Cool', 'Sunny and cold', 'Sunny and warm', 'Sunny and clear']

    rainy = ['Rainy', 'Scattered Showers', 'Showers', 'Cloudy Rain', 'Light Rain', 'Rain shower', 'Rain likely, temps in low 40s.', 'Cloudy, Rain']

    snow = ['Heavy lake effect snow']

    indoor = ['Controlled Climate', 'Indoors', 'N/A Indoor', 'N/A (Indoors)']

    if row.Weather in cloudy:
        return 'Cloudy'

    if row.Weather in indoor:
        return 'Indoor'

    if row.Weather in clear:
        return 'Clear'

    if row.Weather in rainy:
        return 'Rain'

    if row.Weather in snow:
        return 'Snow'

    if row.Weather in ['Cloudy.', 'Heat Index 95', 'Cold']:
        return np.nan

    return row.Weather

def clean_stadiumtype(row):
    if row.StadiumType in ['Bowl', 'Heinz Field', 'Cloudy']:
        return np.nan
    else:
        return row.StadiumType

def clean_play_df(play_df):
    play_df_cleaned = play_df.copy()

    # clean StadiumType
    play_df_cleaned['StadiumType'] = play_df_cleaned['StadiumType'].str.replace(r'Oudoor|Outdoors|Ourdoor|Outddors|Outdor|Outside', 'Outdoor')
    play_df_cleaned['StadiumType'] = play_df_cleaned['StadiumType'].str.replace(r'Indoors|Indoor, Roof Closed|Indoor, Open Roof', 'Indoor')
    play_df_cleaned['StadiumType'] = play_df_cleaned['StadiumType'].str.replace(r'Closed Dome|Domed, closed|Domed, Open|Domed, open|Dome, closed|Domed', 'Dome')
    play_df_cleaned['StadiumType'] = play_df_cleaned['StadiumType'].str.replace(r'Retr. Roof-Closed|Outdoor Retr Roof-Open|Retr. Roof - Closed|Retr. Roof-Open|Retr. Roof - Open|Retr. Roof Closed', 'Retractable Roof')
    play_df_cleaned['StadiumType'] = play_df_cleaned['StadiumType'].str.replace('Open', 'Outdoor')
    play_df_cleaned['StadiumType'] = play_df_cleaned.apply(lambda row: clean_stadiumtype(row), axis=1)

    # clean Weather
    play_df_cleaned['Weather'] = play_df_cleaned.apply(lambda row: clean_weather(row), axis=1)

    return play_df_cleaned

In [ ]:
play_df_cleaned = clean_play_df(play_df)
game_df_cleaned = play_df_cleaned[['GameID', 'StadiumType', 'FieldType', 'Weather', 'Temperature']].drop_duplicates().reset_index().drop(columns=['index'])

In [ ]:
#Vejamos rapidamente as lesões:
print('There are {} injury records in total.'.format(len(injury_df)))

There are 105 injury records in total.


In [ ]:
print('{} unique players injured'.format(len(injury_df.PlayerKey.unique())))

100 unique players injured


In [ ]:
#Isso significa que há jogadores lesionados duas vezes!

print('There are {} PlayKey values missing.'.format(len(injury_df) - injury_df.PlayKey.count()))

There are 28 PlayKey values missing.


In [ ]:
# joined cleaned games dataset and injury dataset
game_injury_df = injury_df.set_index('GameID').join(game_df_cleaned.set_index('GameID'), how = 'outer')

# fill null values for the injury columns with zeros
game_injury_df['DM_M1'] = game_injury_df['DM_M1'].fillna(0).astype(int)
game_injury_df['DM_M7'] = game_injury_df['DM_M7'].fillna(0).astype(int)
game_injury_df['DM_M28'] = game_injury_df['DM_M28'].fillna(0).astype(int)
game_injury_df['DM_M42'] = game_injury_df['DM_M42'].fillna(0).astype(int)

game_injury_df.DM_M1 = game_injury_df.DM_M1 - game_injury_df.DM_M7
game_injury_df.DM_M7 = game_injury_df.DM_M7 - game_injury_df.DM_M28
game_injury_df.DM_M28 = game_injury_df.DM_M28 - game_injury_df.DM_M42

# introduce a column with a flag indicating an injury
game_injury_df['Injury'] = game_injury_df['DM_M1'] + game_injury_df['DM_M7'] + game_injury_df['DM_M28'] + game_injury_df['DM_M42']

# drop duplicated surface column
game_injury_df = game_injury_df.drop(columns=['Surface'])

# drop play-level features just for now
game_injury_df = game_injury_df.drop(columns=['PlayerKey', 'PlayKey'])

# create dummy variables
game_injury_df_dummies = pd.get_dummies(game_injury_df, dummy_na = True, drop_first = True).drop(columns=['FieldType_nan'])

In [ ]:
# joined cleaned games dataset and injury dataset
play_injury_df = injury_df.dropna(subset=['PlayKey']).set_index('PlayKey').join(play_df_cleaned.set_index('PlayKey'), how = 'outer', lsuffix='_left', rsuffix='_right')

# fill null values for the injury columns with zeros
play_injury_df['DM_M1'] = play_injury_df['DM_M1'].fillna(0).astype(int)
play_injury_df['DM_M7'] = play_injury_df['DM_M7'].fillna(0).astype(int)
play_injury_df['DM_M28'] = play_injury_df['DM_M28'].fillna(0).astype(int)
play_injury_df['DM_M42'] = play_injury_df['DM_M42'].fillna(0).astype(int)

# introduce a column with a flag indicating an injury
play_injury_df.DM_M1 = play_injury_df.DM_M1 - play_injury_df.DM_M7
play_injury_df.DM_M7 = play_injury_df.DM_M7 - play_injury_df.DM_M28
play_injury_df.DM_M28 = play_injury_df.DM_M28 - play_injury_df.DM_M42

play_injury_df['Injury'] = play_injury_df['DM_M1'] + play_injury_df['DM_M7'] + play_injury_df['DM_M28']+ play_injury_df['DM_M42']

# drop duplicated surface column
play_injury_df = play_injury_df.drop(columns=['Surface'])

# create dummy variables
play_injury_df_dummies = pd.get_dummies(play_injury_df, columns = ['PlayType', 'PositionGroup'], dummy_na = True, drop_first = True)

In [ ]:
features_df = play_injury_df.copy().reset_index()
features_df = features_df.drop(columns=['PlayerKey_left','GameID_left','BodyPart', 'PlayKey', 'PlayerKey_right', 'GameID_right', 'DM_M1', 'DM_M7', 'DM_M28', 'DM_M42'])

In [ ]:
# reset index
features_df = play_injury_df.copy().reset_index()
# drop the irrelevant columns
features_df = features_df.drop(columns=['PlayerKey_left','GameID_left','BodyPart', 'PlayKey', 'PlayerKey_right', 'GameID_right', 'DM_M1', 'DM_M7', 'DM_M28', 'DM_M42'])
# convert into dummies
features_df = pd.get_dummies(features_df, dummy_na = False, drop_first = True)

In [ ]:
#separa em features e targets (treino)
y = features_df['Injury']
X = features_df.drop(columns=['Injury'])

In [ ]:
#Importar PCA
from sklearn.decomposition import PCA

#Padronizar os dados
##StandardScaler é uma classe do scikit-learn que fornece uma implementação da padronização dos conjuntos de dados. A padronização é uma técnica importante, especialmente quando se utiliza algoritmos que são sensíveis à escala das variáveis, como é frequentemente o caso em modelos baseados em distância, como o k-Nearest Neighbors (k-NN) ou máquinas de vetores de suporte (SVM).
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() #scaler = StandardScaler(): Cria uma instância do StandardScaler. Esta instância será usada para ajustar os parâmetros de padronização (média e desvio padrão) aos dados de treinamento e, em seguida, aplicar a mesma transformação aos dados de teste.

X_padronizado = scaler.fit_transform(X) #X_padronizado = scaler.fit_transform(X_train): Aplica a padronização aos dados (features_df). O método fit_transform ajusta o scaler aos dados de treinamento, calculando a média e o desvio padrão, e, em seguida, aplica a transformação aos dados, resultando em X_padronizado.


In [ ]:
X_padronizado.shape

(267006, 83)

In [ ]:
#Aplicar o PCA

num_componentes = 30  # ajuste conforme necessário
pca = PCA(n_components=num_componentes)
X_pca = pca.fit_transform(X_padronizado)


#X_pca = pca.fit_transform(X_padronizado) #X_pca =
#pca.fit_transform(X_padronizado): Ajusta o modelo PCA aos dados
#padronizados de treinamento (X_padronizado). Durante esse
#processo, o PCA calcula os componentes principais
#(combinações lineares das características originais) que explicam
#a maior parte da variância nos dados.
print("Original shape: {}".format(str(X.shape)))
print("Reduced shape: {}".format(str(X_pca.shape)))

Original shape: (267006, 83)
Reduced shape: (267006, 30)


In [ ]:
X_pca = pd.DataFrame(X_pca)

In [ ]:
display(y)

0         0
1         0
2         0
3         0
4         0
         ..
267001    0
267002    0
267003    0
267004    0
267005    0
Name: Injury, Length: 267006, dtype: int64

In [ ]:
#juntando o dataset de X_pca e y, para formar novamente um dataset só e aí sim o balancear
frames = [X_pca, y]
new_features_df = pd.concat(frames, axis = 1)
new_features_df.reset_index(inplace=True)
new_features_df = new_features_df.drop(columns=['index'])

In [ ]:
display(new_features_df)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Injury
0,0.175595,0.866855,0.562708,-0.146938,6.727808,7.681756,2.102534,-0.764971,-1.371724,2.035850,...,0.137381,-0.237264,-0.281913,-0.006660,0.160051,-0.154241,0.117276,-0.280918,-0.037194,0
1,0.176635,0.910490,0.516429,-0.174252,6.737365,7.698218,2.089047,-0.766451,-1.352118,2.040369,...,0.100538,-0.212434,-0.266415,-0.038516,0.131989,-0.102481,0.112000,-0.239339,0.048900,0
2,0.026375,0.913497,0.555040,-0.160741,6.672772,7.772822,2.133262,-0.757104,-1.391643,2.028414,...,0.125017,-0.187253,-0.306760,0.003458,0.137331,-0.067831,0.101514,-0.179521,0.148192,0
3,0.176866,0.920187,0.506145,-0.180322,6.739489,7.701876,2.086049,-0.766780,-1.347761,2.041373,...,0.092350,-0.206917,-0.262971,-0.045595,0.125753,-0.090979,0.110828,-0.230099,0.068032,0
4,0.176982,0.925035,0.501002,-0.183357,6.740551,7.703705,2.084551,-0.766944,-1.345583,2.041875,...,0.088257,-0.204158,-0.261250,-0.049135,0.122635,-0.085228,0.110242,-0.225479,0.077598,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267001,-0.051073,0.315291,-0.021284,0.162957,0.074671,0.018421,0.076286,-1.169942,1.130885,1.099600,...,1.432864,-1.187495,-1.391548,1.935368,-1.831841,2.301588,1.621139,-2.056498,4.392605,0
267002,-0.056622,0.082570,0.225540,0.308631,0.023700,-0.069377,0.148218,-1.162050,1.026321,1.075499,...,1.629362,-1.319922,-1.474200,2.105265,-1.682178,2.025537,1.649274,-2.278254,3.933439,0
267003,-0.206882,0.085577,0.264151,0.322143,-0.040893,0.005227,0.192434,-1.152704,0.986797,1.063545,...,1.653841,-1.294740,-1.514545,2.147239,-1.676835,2.060188,1.638788,-2.218436,4.032732,0
267004,-0.056391,0.092267,0.215255,0.302562,0.025823,-0.065719,0.145221,-1.162379,1.030678,1.076503,...,1.621174,-1.314404,-1.470756,2.098186,-1.688414,2.037040,1.648102,-2.269014,3.952571,0


In [ ]:
#separando o new_features_df em 2 datasets, um com injury igual a 0 e o outro com injury igual a 1_
new_features_df0 = new_features_df[new_features_df['Injury'] < 1]
new_features_df1 = new_features_df[new_features_df['Injury'] >= 1]

In [ ]:
#selecionando aleatoriamente 770 casos de não lesão
new_features_df770 = new_features_df0.sample(n=770)

In [ ]:
#multilplicando o dataframe de lesões por 10
new_features_df2 = pd.DataFrame(np.repeat(new_features_df1.values, 10, axis=0))
new_features_df2.columns = new_features_df1.columns

In [ ]:
#juntando o dataset de injury = 1 com as 770 amostras de treino
new_frames = [new_features_df2, new_features_df770]
post_pca = pd.concat(new_frames)
post_pca.reset_index(inplace=True)
post_pca = post_pca.drop(columns=['index'])

In [ ]:
#separa em features e targets (treino)
post_y = post_pca['Injury']
post_X = post_pca.drop(columns=['Injury'])

In [ ]:
skf = StratifiedKFold(n_splits=2, shuffle = True)
#usa separação estratificada para treinar o conjunto de treino para distribuir igualmente os rótulos em todas as classes
for train_index, test_index in skf.split(post_X, post_y):
    X_train, X_teste = post_X.values[train_index, :], post_X.values[test_index, :]
    y_train, y_teste = post_y[train_index], post_y[test_index]

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter = 100000).fit(X_train, y_train)
print("Training set score: {:.3f}".format(logreg.score(X_train, y_train)))
print("Test set score: {:.3f}".format(logreg.score(X_teste, y_teste)))

y_pred = logreg.predict(X_teste)
f1 = f1_score(y_pred, y_teste, average="weighted")
print("F1 Score:", f1)

Training set score: 0.687
Test set score: 0.656
F1 Score: 0.6558493800803096


In [ ]:
logreg100 = LogisticRegression(C=100, max_iter = 100000).fit(X_train, y_train)
print("Training set score: {:.3f}".format(logreg100.score(X_train, y_train)))
print("Test set score: {:.3f}".format(logreg100.score(X_teste, y_teste)))

y_pred = logreg.predict(X_teste)
f1 = f1_score(y_pred, y_teste, average="weighted")
print("F1 Score:", f1)

Training set score: 0.691
Test set score: 0.664
F1 Score: 0.6558493800803096


In [ ]:
logreg001 = LogisticRegression(C=0.01, max_iter = 100000).fit(X_train, y_train)
print("Training set score: {:.3f}".format(logreg001.score(X_train, y_train)))
print("Test set score: {:.3f}".format(logreg001.score(X_teste, y_teste)))

y_pred = logreg.predict(X_teste)
f1 = f1_score(y_pred, y_teste, average="weighted")
print("F1 Score:", f1)

Training set score: 0.697
Test set score: 0.661
F1 Score: 0.6558493800803096


In [ ]:
plt.plot(logreg.coef_.T, 'o', label="C=1")
plt.plot(logreg100.coef_.T, '^', label="C=100")
plt.plot(logreg001.coef_.T, 'v', label="C=0.001")
plt.xticks(range(features_df.shape[1]), list(features_df.columns.values), rotation=90)
plt.hlines(0, 0, features_df.shape[1])
plt.ylim(-5, 5)
plt.xlabel("Coefficient index")
plt.ylabel("Coefficient magnitude")
plt.legend()

[[251 134]
 [131 254]]


In [ ]:
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_teste)

from sklearn.metrics import confusion_matrix

labels = ["(Não Lesão)", "(Lesão)"]
cf_matrix = confusion_matrix(y_teste, y_pred)
print(cf_matrix)

import seaborn as sns
ax= plt.subplot()
sns.heatmap(cf_matrix, annot=True, fmt='g', ax=ax, cmap = 'Blues')

ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Matriz de Confusão - RL/PCA');
ax.xaxis.set_ticklabels(['Não Lesão', 'Lesão']); ax.yaxis.set_ticklabels(['Não Lesão', 'Lesão']);